[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np 

In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv('https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv', sep='\t', encoding='latin-1')

In [3]:
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [93]:
# stop words
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import re

In [12]:
emoticon_pattern1 = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'
emoticon_pattern2 = '(?::|;|=)(?:-)?(?:\)|\(|D|P)'
emoticons = df.review.apply(lambda x: re.findall(emoticon_pattern2, x))
emoticons.head()

0    []
1    []
2    []
3    []
4    []
Name: review, dtype: object

In [90]:
# Removing special characters and "trash"
def my_preprocessor(text):
    # Remove HTML markup
    # Your code here
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    # Your code here
    emoticon_pattern = '(?::+|;|=+)(?:-)?(?:\(+|\)+|D|P|d|p)'
    icons = re.findall(emoticon_pattern, text)
    text = re.sub('[\s]+'+emoticon_pattern, '', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (text + ' ' + ' '.join(icons))
    return text

print(my_preprocessor('nguyen :) :D ::-)))) ===))))) :))) :p tien phat'))

nguyen tien phat :) :D ::-)))) ===))))) :))) :p


In [95]:
def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticon_pattern = '(?::|;|=)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\s]+'+emoticon_pattern, '', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

# Create some random texts for testing the function preprocessor()
print(preprocessor('nguyen 1 2 3 :-( :D :))) =)))) :P tien phat'))

nguyen 1 2 3 tien phat :( :D :) =) :P


In [91]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # Your code here

In [92]:
# split the dataset in train and test
# Your code here

from sklearn.model_selection import train_test_split

X=df.review
y=df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [96]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                       tokenizer=tokenizer_porter,
                       preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
               ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/anaconda3/envs/machine/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))
/anaconda3/envs/machine/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [97]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

predictions = clf.predict(X_test)
print('accuracy score: ', accuracy_score(y_test, predictions))
print('confusion matrix:\n', confusion_matrix(y_test, predictions))
print('classification report:\n', classification_report(y_test, predictions))

accuracy score:  0.8841333333333333
confusion matrix:
 [[3197  500]
 [ 369 3434]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      3697
           1       0.87      0.90      0.89      3803

   micro avg       0.88      0.88      0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500



Try some tests

In [99]:
twits = [
    "This is really bad",
    "I love this!",
    ":)",
]

predicts = clf.predict_proba(twits)
for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {predicts[i]}')

This is really bad --> Negative, Positive = [0.99462391 0.00537609]
I love this! --> Negative, Positive = [0.02648923 0.97351077]
:) --> Negative, Positive = [0.17584384 0.82415616]


## 5. Export Model 

In [102]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open('W06_Assignment.pkl', 'wb+'), protocol=4)